# Mount Azure Data Lake using Service Principal
1. Register Azure AD Application / Service Principal
2. Generate a secret/password for the application
3. Set Spark Config with App/Client Id, Directory/Tenant Id, Secret
4. Assign role "Storage Blob Data Contributor" to the Data Lake
5. Call file system utility mount to mount the storage to the Databricks workspace

In [0]:
def mount_adls(storage_account_name, container_name):
    # Get secrets from Key Vault
    client_id = dbutils.secrets.get(scope = 'f1project-scope', key = 'f1-client-id')
    tenant_id = dbutils.secrets.get(scope = 'f1project-scope', key = 'f1-tenant-id')
    client_secret = dbutils.secrets.get(scope = 'f1project-scope', key = 'f1-client-secret')

    # Set Spark configurations
    configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": client_id,
           "fs.azure.account.oauth2.client.secret": client_secret,
           "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    # Unmount the mount point if it already exists
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")

    # Mount the storage account container
    dbutils.fs.mount(
        source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
        mount_point = f"/mnt/{storage_account_name}/{container_name}",
        extra_configs = configs)
    
    display(dbutils.fs.mounts())

In [0]:
mount_adls('databricksf1projectdl', 'raw')

/mnt/databricksf1projectdl/raw has been unmounted.


mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/mnt/databricksf1projectdl/processed,abfss://processed@databricksf1projectdl.dfs.core.windows.net/,
/mnt/databricksf1projectdl/presentation,abfss://presentation@databricksf1projectdl.dfs.core.windows.net/,
/mnt/databricksf1projectdl/raw,abfss://raw@databricksf1projectdl.dfs.core.windows.net/,
/Volume,DbfsReserved,
/volumes,DbfsReserved,


In [0]:
mount_adls('databricksf1projectdl', 'processed')

/mnt/databricksf1projectdl/processed has been unmounted.


mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/mnt/databricksf1projectdl/processed,abfss://processed@databricksf1projectdl.dfs.core.windows.net/,
/mnt/databricksf1projectdl/presentation,abfss://presentation@databricksf1projectdl.dfs.core.windows.net/,
/mnt/databricksf1projectdl/raw,abfss://raw@databricksf1projectdl.dfs.core.windows.net/,
/Volume,DbfsReserved,
/volumes,DbfsReserved,


In [0]:
mount_adls('databricksf1projectdl', 'presentation')

/mnt/databricksf1projectdl/presentation has been unmounted.


mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/mnt/databricksf1projectdl/processed,abfss://processed@databricksf1projectdl.dfs.core.windows.net/,
/mnt/databricksf1projectdl/presentation,abfss://presentation@databricksf1projectdl.dfs.core.windows.net/,
/mnt/databricksf1projectdl/raw,abfss://raw@databricksf1projectdl.dfs.core.windows.net/,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
